In [29]:
%load_ext autoreload
%autoreload 3 --log
%autoreload?

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Docstring:
::

  %autoreload [-p] [-l] [--hide-errors] [mode]

%autoreload => Reload modules automatically

%autoreload or %autoreload now
Reload all modules (except those excluded by %aimport) automatically
now.

%autoreload 0 or %autoreload off
Disable automatic reloading.

%autoreload 1 or %autoreload explicit
Reload only modules imported with %aimport every time before executing
the Python code typed.

%autoreload 2 or %autoreload all
Reload all modules (except those excluded by %aimport) every time
before executing the Python code typed.

%autoreload 3 or %autoreload complete
Same as 2/all, but also but also adds any new objects in the module. See
unit test at IPython/extensions/tests/test_autoreload.py::test_autoload_newly_added_objects

The optional arguments --print and --log control display of autoreload activity. The default
is to act silently; --print (or -p) will print out the names of modules that are being
reloaded, and --log (or -l) outputs them to the log at INFO level.

In [30]:
import torch

if torch.cuda:
    device = torch.device("cuda")
    torch.set_default_device(device)
else:
    device = torch.device("cpu")

torch.set_default_dtype(torch.float64)

print(f"Using device: {device}")

Using device: cpu


In [31]:
# Define Data

from utils import *
from stpy.borel_set import BorelSet

name = "sensepy/sensepy/benchmarks/data/taxi_data.csv"
Num_data_points = 200
# borel set a square with boundaries [-1,1]^2
domain = BorelSet(2, bounds=torch.tensor([[-1.0, 1.0], [-1.0, 1.0]]).double())
obs, dt, gdf = get_taxi_data(Num_data_points)

In [32]:
# Define the Process

from functools import partial
import numpy as np
import scipy
from stpy.kernels import KernelFunction
from tqdm import tqdm
from autograd_minimize import minimize
import torch
from typing import List
from naive_ppp_estimator import NaivePPPEstimator


kernel_object = KernelFunction(kernel_name="squared_exponential", gamma=0.12, d=2)
process = NaivePPPEstimator(kernel_object)

In [33]:
# MAP by representer theorem with two areas

process.load_data(
    domain, obs
)

In [34]:
left, down, right, up = gdf.total_bounds
n = 30  # discretization
k = 3  # number of samples to draw and plot

xtest = domain.return_discretization(n)
Map = BorelSet(d=2, bounds=torch.tensor([[left, right], [down, up]]).double())
xtest_orig = Map.return_discretization(n).cpu().numpy()

xx = xtest_orig[:, 0]
yy = xtest_orig[:, 1]

In [39]:
intensity = process.fit(xtest)

KeyboardInterrupt: 

In [ ]:
# Plotting
import matplotlib.pyplot as plt
import contextily as ctx

fig, ax = plt.subplots(figsize=(10, 10))

# Define a scaling factor for the sizes based on mean values
scale_factor = 100  # Adjust this factor as needed for visual clarity

# Scatter plot with mean values as size and std as color using viridis colormap
scatter = ax.scatter(
    xx,
    yy,
    s=intensity * scale_factor,  # Scale point size by mean values
    alpha=1.0,
    edgecolor="face",
    linewidth=0.0,
    marker="o",
)

ctx.add_basemap(ax, crs=gdf.crs.to_string())

# Labeling
ax.set_title("MAP of intensity")
ax.set_xlabel("X-axis")
ax.set_ylabel("Y-axis")

plt.show()

In [24]:
print(
    f"Mean in [{torch.min(mean_values)}, {torch.max(mean_values)}] and std in"
    f" [{torch.min(std_values)}, {torch.max(std_values)}]"
)

Mean in [0.04894289033170558, 1.2456873777126345] and std in [0.0, 0.0]


In [25]:
map_integral = process.rate_value(xtest, dt).squeeze(1).mean() * 4.0
map_integral

tensor(328.4789, device='cuda:0')

In [26]:
print(f"Got expected num points = {map_integral} while actual points is {len(obs)}")

Got expected num points = 328.47892979552665 while actual points is 200
